In [9]:
# Probar con arbol con poda
# Probar con arbol sin poda pero corto
# Probar voting
# Probar varios KFolds en el de stacking

#Agregamos librerias a utilizar
import keras as keras
import pandas as pd
import xgboost as xgb
# import sklearn as sk
import numpy as np
from datetime import datetime

#Visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt

#modelos y métricas
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split, cross_validate
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import StackingRegressor,RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
pd.set_option('display.max_columns', 40)

# import warnings
# warnings.simplefilter("ignore")

In [10]:
df =  pd.read_csv('train.csv')

In [14]:
# Generamos un dataframe del dataset de entrenamiento
df =  pd.read_csv('train.csv')
df_final = df.drop(["shutdown", "mini_shutdown", "frankfurt_shutdown", "precipitation", "snow", "blackout", "mov_change", 'id'], axis = 1)
df_final['date'] = pd.to_datetime(df_final['date'])
df_final = df_final.sort_values(by = "date")
df_final_label = df_final.user_activity_2
#df_final = df_final.drop(["orders"], axis=1)

def get_season(fecha):
    if fecha in ['December', 'January', 'February']:
        return 'Invierno'
    elif fecha in ['March', 'April', 'May']:
        return 'Primavera'
    elif fecha in ['June', 'July', 'August']:
        return 'Verano'
    elif fecha in ['September', 'October', 'November']:
        return 'Otoño'

# Separando el DataFrame en 80% y 20%
df_final_train, df_final_validation, df_final_train_label, df_final_validation_label = train_test_split(df_final,
                                                    df_final_label, 
                                                    test_size=0.2,
                                                    shuffle=False)
warehouses = df_final_train.warehouse
# mean user activity 1
df_usact1 = df_final_train[["user_activity_1", 'warehouse']]
df_usact1 = df_usact1.groupby("warehouse")["user_activity_1"].mean()
means_ua1= df_usact1.to_dict()

PROMEDIO_UA1 = df_usact1.mean()

# max user activity 1
df_usact1 = df_final_train[["user_activity_1", 'warehouse']]
df_usact1 = df_usact1.groupby("warehouse")["user_activity_1"].max()
max_ua1= df_usact1.to_dict()

# min user activity 1
df_usact1 = df_final_train[["user_activity_1", 'warehouse']]
df_usact1 = df_usact1.groupby("warehouse")["user_activity_1"].min()
min_ua1= df_usact1.to_dict()
# user activity 2
df_usact2 = df_final_train[["user_activity_2", 'warehouse']]
df_usact2 = df_usact2.groupby("warehouse")["user_activity_2"].mean()
PROMEDIO_UA2 = df_usact1.mean()
means_ua2= df_usact2.to_dict()

# max user activity 1
df_usact2 = df_final_train[["user_activity_2", 'warehouse']]
df_usact2 = df_usact2.groupby("warehouse")["user_activity_2"].max()
max_ua2= df_usact2.to_dict()

# min user activity 1
df_usact2 = df_final_train[["user_activity_2", 'warehouse']]
df_usact2 = df_usact2.groupby("warehouse")["user_activity_2"].min()
min_ua2= df_usact2.to_dict()

df_mean_encoding_warehouse = pd.concat([df_final_train, df_final_train_label], axis = 1)[["orders", 'warehouse']]
df_mean_encoding_warehouse = df_mean_encoding_warehouse.groupby("warehouse")["orders"].mean()
PROMEDIO_OPW = df_mean_encoding_warehouse.mean()
means_orders_per_warehouse= df_mean_encoding_warehouse.to_dict()
df_final_train['year_month_warehouse'] = df_final_train['date'].dt.month.astype(str) + '_' + df_final_train['warehouse']
df_mean_encoding_year_month_warehouse = pd.concat([df_final_train, df_final_train_label], axis = 1)[["orders", 'year_month_warehouse']]
df_mean_encoding_year_month_warehouse = df_mean_encoding_year_month_warehouse.groupby('year_month_warehouse')["orders"].mean()
PROMEDIO_YMW = df_mean_encoding_year_month_warehouse.mean()
means_y_m_w= df_mean_encoding_year_month_warehouse.to_dict()
#Features relacionadas con la user activity
df_final_train["mean_user_activity_1"] = df_final_train.warehouse.apply(lambda x: means_ua1.get(x, PROMEDIO_UA1))
df_final_train["mean_user_activity_2"] = df_final_train.warehouse.apply(lambda x: means_ua2.get(x, PROMEDIO_UA2))
df_final_train["max_user_activity_1"] = df_final_train.warehouse.apply(lambda x: max_ua1.get(x, PROMEDIO_UA1))
df_final_train["max_user_activity_2"] = df_final_train.warehouse.apply(lambda x: max_ua2.get(x, PROMEDIO_UA2))
df_final_train["min_user_activity_1"] = df_final_train.warehouse.apply(lambda x: min_ua1.get(x, PROMEDIO_UA1))
df_final_train["min_user_activity_2"] = df_final_train.warehouse.apply(lambda x: min_ua2.get(x, PROMEDIO_UA2))
df_final_train["product_ua"] = df_final_train["mean_user_activity_1"] * df_final_train["mean_user_activity_2"]


#Mean encoding
df_final_train["mean_orders_per_warehouse"] = df_final_train.warehouse.apply(lambda x: means_orders_per_warehouse.get(x, PROMEDIO_OPW))
df_final_train['year_month_warehouse'] = df_final_train['date'].dt.month.astype(str) + '_' + df_final_train['warehouse']
df_final_train["means_year_month_warehouse"] = df_final_train.year_month_warehouse.apply(lambda x: means_y_m_w.get(x, PROMEDIO_YMW))

city_to_country = {
    'Munich_1': 'Germany',
    'Frankfurt_1': 'Germany',
    'Budapest_1': 'Hungary',
    'Brno_1': 'Czech Republic',
    'Prague_1': 'Czech Republic',
    'Prague_2': 'Czech Republic',
    'Prague_3': 'Czech Republic'
}

#Countries and warehouses
df_final_train['country'] = df_final_train['warehouse'].map(city_to_country)
df_final_train = pd.concat([df_final_train.drop(["country"], axis=1), pd.get_dummies(df_final_train.country).astype('int').add_prefix('country_')], axis=1)
df_final_train = pd.concat([df_final_train.drop(["warehouse"], axis=1), pd.get_dummies(df_final_train.warehouse).astype('int').add_prefix('warehouse_')], axis=1)
df_final_train = df_final_train.drop(["warehouse_Budapest_1", "warehouse_Prague_2", "warehouse_Prague_3"], axis=1)
df_final_train['hot_season'] = df_final_train.date.apply(lambda x: 1 if 2 < x.month < 9 else 0)

#Time
df_final_train['year'] = df_final_train['date'].dt.year
df_final_train['month'] = df_final_train['date'].dt.month
df_final_train['week'] = df_final_train['date'].dt.isocalendar().week
df_final_train['day'] = df_final_train['date'].dt.day
df_final_train['day_of_week'] = df_final_train['date'].dt.dayofweek #numero
# df_final_train['day_of_week'] = df_final_train['date'].dt.day_name() #cadena
df_final_train['year_sin'] = np.sin(2 * np.pi * df_final_train['year'])
#df_final_train['year_cos'] = np.cos(2 * np.pi * df_final_train['year'])
#df_final_train['month_sin'] = np.sin(2 * np.pi * df_final_train['month'] / 12) 
df_final_train['month_cos'] = np.cos(2 * np.pi * df_final_train['month'] / 12)
#df_final_train['week_sin'] = np.sin(2 * np.pi * df_final_train['week'] / 53)
df_final_train['week_cos'] = np.cos(2 * np.pi * df_final_train['week'] / 53)
#df_final_train['day_sin'] = np.sin(2 * np.pi * df_final_train['day'] / 31)
#df_final_train['day_cos'] = np.cos(2 * np.pi * df_final_train['day'] / 31)
#df_final_train['day_of_week_sin'] = np.sin(2 * np.pi * df_final_train['day_of_week'] / 7)
#df_final_train['day_of_week_cos'] = np.cos(2 * np.pi * df_final_train['day_of_week'] / 7)
df_final_train["month_name"] = df_final_train.date.dt.month_name()
df_final_train['season'] = df_final_train.month_name.apply(get_season)
df_final_train = pd.concat([df_final_train, pd.get_dummies(df_final_train.season).astype('int').add_prefix('Season_')], axis=1)
df_final_train['group']=(df_final_train['year']-2020)*48+df_final_train['month']*4+df_final_train['day']//7

#drops
df_final_train = df_final_train.drop(['date', 'year', 'month', 'week', 'day', 'day_of_week', 'month_name', 'season', 'holiday_name', 'year_month_warehouse', 'user_activity_1', 'user_activity_2'], axis = 1)
df_final_train = df_final_train.reset_index().drop(["index"], axis = 1)

warehouses_validation = df_final_validation.warehouse
#Features relacionadas con la user activity
df_final_validation["mean_user_activity_1"] = df_final_validation.warehouse.apply(lambda x: means_ua1.get(x, PROMEDIO_UA1))
df_final_validation["mean_user_activity_2"] = df_final_validation.warehouse.apply(lambda x: means_ua2.get(x, PROMEDIO_UA2))
df_final_validation["max_user_activity_1"] = df_final_validation.warehouse.apply(lambda x: max_ua1.get(x, PROMEDIO_UA1))
df_final_validation["max_user_activity_2"] = df_final_validation.warehouse.apply(lambda x: max_ua2.get(x, PROMEDIO_UA2))
df_final_validation["min_user_activity_1"] = df_final_validation.warehouse.apply(lambda x: min_ua1.get(x, PROMEDIO_UA1))
df_final_validation["min_user_activity_2"] = df_final_validation.warehouse.apply(lambda x: min_ua2.get(x, PROMEDIO_UA2))
df_final_validation["product_ua"] = df_final_validation["mean_user_activity_1"] * df_final_validation["mean_user_activity_2"]


#Mean encoding
df_final_validation["mean_orders_per_warehouse"] = df_final_validation.warehouse.apply(lambda x: means_orders_per_warehouse.get(x, PROMEDIO_OPW))
df_final_validation['year_month_warehouse'] = df_final_validation['date'].dt.month.astype(str) + '_' + df_final_validation['warehouse']
df_final_validation["means_year_month_warehouse"] = df_final_validation.year_month_warehouse.apply(lambda x: means_y_m_w.get(x, PROMEDIO_YMW))

city_to_country = {
    'Munich_1': 'Germany',
    'Frankfurt_1': 'Germany',
    'Budapest_1': 'Hungary',
    'Brno_1': 'Czech Republic',
    'Prague_1': 'Czech Republic',
    'Prague_2': 'Czech Republic',
    'Prague_3': 'Czech Republic'
}

#Countries and warehouses
df_final_validation['country'] = df_final_validation['warehouse'].map(city_to_country)
df_final_validation = pd.concat([df_final_validation.drop(["country"], axis=1), pd.get_dummies(df_final_validation.country).astype('int').add_prefix('country_')], axis=1)
df_final_validation = pd.concat([df_final_validation.drop(["warehouse"], axis=1), pd.get_dummies(df_final_validation.warehouse).astype('int').add_prefix('warehouse_')], axis=1)
df_final_validation = df_final_validation.drop(["warehouse_Budapest_1", "warehouse_Prague_2", "warehouse_Prague_3"], axis=1)
df_final_validation['hot_season'] = df_final_validation.date.apply(lambda x: 1 if 2 < x.month < 9 else 0)

#Time
df_final_validation['year'] = df_final_validation['date'].dt.year
df_final_validation['month'] = df_final_validation['date'].dt.month
df_final_validation['week'] = df_final_validation['date'].dt.isocalendar().week
df_final_validation['day'] = df_final_validation['date'].dt.day
df_final_validation['day_of_week'] = df_final_validation['date'].dt.dayofweek #numero
# df_final_validation['day_of_week'] = df_final_validation['date'].dt.day_name() #cadena
df_final_validation['year_sin'] = np.sin(2 * np.pi * df_final_validation['year'])
#df_final_validation['year_cos'] = np.cos(2 * np.pi * df_final_validation['year'])
#df_final_validation['month_sin'] = np.sin(2 * np.pi * df_final_validation['month'] / 12) 
df_final_validation['month_cos'] = np.cos(2 * np.pi * df_final_validation['month'] / 12)
#df_final_validation['week_sin'] = np.sin(2 * np.pi * df_final_validation['week'] / 53)
df_final_validation['week_cos'] = np.cos(2 * np.pi * df_final_validation['week'] / 53)
#df_final_validation['day_sin'] = np.sin(2 * np.pi * df_final_validation['day'] / 31)
#df_final_validation['day_cos'] = np.cos(2 * np.pi * df_final_validation['day'] / 31)
#df_final_validation['day_of_week_sin'] = np.sin(2 * np.pi * df_final_validation['day_of_week'] / 7)
#df_final_validation['day_of_week_cos'] = np.cos(2 * np.pi * df_final_validation['day_of_week'] / 7)
df_final_validation["month_name"] = df_final_validation.date.dt.month_name()
df_final_validation['season'] = df_final_validation.month_name.apply(get_season)
df_final_validation = pd.concat([df_final_validation, pd.get_dummies(df_final_validation.season).astype('int').add_prefix('Season_')], axis=1)
df_final_validation['group']=(df_final_validation['year']-2020)*48+df_final_validation['month']*4+df_final_validation['day']//7

#drops
df_final_validation = df_final_validation.drop(['date', 'year', 'month', 'week', 'day', 'day_of_week', 'month_name', 'season', 'holiday_name', 'year_month_warehouse', 'user_activity_1', 'user_activity_2'], axis = 1)
df_final_validation = df_final_validation.reset_index().drop(["index"], axis = 1)

df_for_corr = pd.concat([df_final_train, df_final_train_label], axis = 1).corr()
features_orders  = df_for_corr.sort_values(by="orders", ascending=False).orders
best_features = list(features_orders[features_orders > 0.1].index)
best_features.pop(0)
# Reordenar las columnas del DataFrame según el orden deseado
df_final_train = df_final_train[best_features]
df_final_validation = df_final_validation[best_features]

In [ ]:
df_final_train

,year_sin,week_cos,month_cos,Season_Invierno,means_year_month_warehouse,min_user_activity_2,mean_orders_per_warehouse,mean_user_activity_2,max_user_activity_2,country_Czech Republic,product_ua,min_user_activity_1,mean_user_activity_1
0,-8.073961e-13,0.889657,1.0,1,8767.872093,30403.0,8261.953704,35952.602881,41952.0,1,6.017289e+07,1564.0,1673.672840
1,-8.073961e-13,0.889657,1.0,1,5268.209302,17799.0,4982.338477,20996.281893,24401.0,1,2.737446e+07,1248.0,1303.776749
2,-8.073961e-13,0.889657,1.0,1,4864.523256,16254.0,4576.633094,18871.905447,21411.0,1,1.804961e+07,914.0,956.427544
3,-8.073961e-13,0.889657,1.0,1,5482.445783,17478.0,5362.533546,21435.602769,24488.0,0,6.409022e+07,2921.0,2989.895634
4,-8.073961e-13,0.889657,1.0,1,7309.081395,25037.0,7009.977366,29416.394033,34040.0,1,6.837638e+07,2245.0,2324.431070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,-6.447061e-13,-0.861044,-0.5,0,7320.764706,30403.0,8261.953704,35952.602881,41952.0,1,6.017289e+07,1564.0,1673.672840
5868,-6.447061e-13,-0.861044,-0.5,0,4983.166667,17478.0,5362.533546,21435.602769,24488.0,0,6.409022e+07,2921.0,2989.895634
5869,-6.447061e-13,-0.861044,-0.5,0,4504.602941,17799.0,4982.338477,20996.281893,24401.0,1,2.737446e+07,1248.0,1303.776749
5870,-6.447061e-13,-0.794854,-0.5,0,4095.637681,16254.0,4576.633094,18871.905447,21411.0,1,1.804961e+07,914.0,956.427544


In [15]:
# Crear el regressor XGBoost
xgb_reg = xgb.XGBRegressor()

# Definir los parámetros para la búsqueda en cuadrícula
param_grid = {
    'max_depth': [2,3, 6, 9, 11, 13, 15],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1],
    'n_estimators': [50, 65, 80, 100, 200, 300],
    'colsample_bytree': [0.3, 0.6, 0.7, 1]
}
best_model = xgb.XGBRegressor(
    colsample_bytree=0.6,
    learning_rate=0.7,
    max_depth=2,
    n_estimators=200
)

best_model.fit(df_final_train, df_final_train_label)
# Hacer predicciones en el conjunto de prueba
y_pred = np.round(best_model.predict(df_final_validation))

# Evaluar el rendimiento del modelo
mse = mean_absolute_percentage_error(df_final_validation_label, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

Mean Squared Error: 0.17


In [16]:
df_final_train_label

0       32575.0
2386    18462.0
3579    17693.0
6186    17840.0
1193    27392.0
         ...   
971     36209.0
7124    22432.0
3357    21491.0
4551    19482.0
5373    16956.0
Name: user_activity_2, Length: 5872, dtype: float64

In [24]:
y_pred

array([19102., 26925.,  5845., ..., 37788., 21909.,  5249.], dtype=float32)

In [35]:
pd.DataFrame({"user_activity_2":y_pred}).to_csv("pred_ua2.csv", index=False)